In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [ ]:
def preprocess_text(text):
    text=text.lower()
    words=word_tokenize(text)
    stop_word=set(stopwords.words('english'))
    words=[word for word in words if word not in stop_word]
    stemmer=PorterStemmer()
    words=[stemmer.stem(word) for word in words]

    proprocessed_text=' '.join(words)
    return proprocessed_text

In [ ]:
import pandas as pd
try:
    ds=pd.read_csv('D:/My WorkSpace/Summer-Internship/05-August-2024/questions_answers.csv',encoding='ISO-8859-1')
    print(ds.head(2))
except UnicodeDecodeError as err:
    print(f"Error: {err}")

In [ ]:
ds.isnull().sum()

In [ ]:
ds.fillna("Not Mentioned",inplace=True)

In [ ]:
ds['ans1']=ds['ans1'].apply(preprocess_text)
ds.head(1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
model=SentenceTransformer('paraphrase-MiniLM-L6-v2')
    

In [ ]:
def suggest_sections(ans,ds,min_suggestions=5):
    preprocessed_ans=preprocess_text(ans)
    ans_embedding=model.encode(preprocessed_ans)
    section_embedding=model.encode(ds['ans1'].tolist())
    similarities=util.pytorch_cos_sim(ans_embedding,section_embedding)[0]
    similarity_threhold=0.2
    relevant_indices=[]
    while len(relevant_indices)<min_suggestions and similarity_threhold>0:
        relevant_indices=[i for i, sim in enumerate(similarities)if sim>similarity_threhold]
        similarity_threhold-=0.5 #st=st-0.5
        sorted_indices=sorted(relevant_indices,key=lambda i: similarities[i],reverse=True)
        suggestions=[
            {
                'index':i,
                'question':ds.iloc[i]['question'],
                'ans':ds.iloc[i]['ans'],
                'similarity_score':similarities[i].items()
            }

            for i in sorted_indices
        ]
    return suggestions


In [ ]:
ans='Python is general purpose programming language '
suggestions=suggest_sections(ans,ds,min_suggestions=1)
for suggestion in suggestions:
        print(f"S_no: {suggestion['index']}")
        print(f"Question: {suggestion['question']}")
        print(f"Your answer : {ans}")
        print(f"Answer: {suggestion['ans']}")
        
        print("_________________________________________________________________________________________\n")
else:
    print("No record is found")
